In [85]:
#Walmart Trip Type, Final Score on LB: 0.66992
import numpy as np
import pandas as pd
# special IPython command to prepare the notebook for matplotlib
#%matplotlib inline 
#import matplotlib as mpl
#import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedShuffleSplit
import xgboost as xgb

In [86]:
traindata = pd.read_csv('train.csv')
traindata = traindata.fillna(0) ## Impute 0
traindata['DepartmentDescription'] = traindata['DepartmentDescription'].replace(to_replace=["MENS WEAR"],value="MENSWEAR")
traindata['DepartmentDescription'] = traindata['DepartmentDescription'].replace(to_replace=["OPTICAL - FRAMES"],value="OPTICAL - LENSES")

#Extract first digit of the UPC
traindata['Upc'] = traindata['Upc'].astype(np.str_)
listupc = list(traindata['Upc'])
traindata['upcfirst'] = [x[0] for x in listupc]

#2 Grams
N2grams = traindata[['VisitNumber','Upc']]
N2gramsgrp = N2grams.groupby(['VisitNumber'])['Upc'].apply(lambda x: "{%s}" % ', '.join(x)).to_frame()
corpusarray = list(N2gramsgrp['Upc'])
vectorizer = TfidfVectorizer(tokenizer=None, 
                             use_idf=False,norm=None,
                             max_features = 300,ngram_range=(2,2)) 
train_data_features = vectorizer.fit_transform(corpusarray)   
train_data_features = train_data_features.A #toarray()  
vocab = vectorizer.get_feature_names()
df2gram = pd.DataFrame()
df2gram = pd.DataFrame(data=train_data_features,columns=vocab)

#Round up Fine line numbers
df = traindata['FinelineNumber'].to_frame()
binsize = 20
labels = [ "{0}".format(i+10) for i in range(0, 10000, binsize) ]
traindata['FinelineNumber'] = pd.cut(df.FinelineNumber, range(0, 10000+binsize, binsize), right=False, labels=labels)

#Dept Dummies
deptDummies = pd.get_dummies(traindata[['DepartmentDescription']],prefix='Dept')
#Get Fineline Dummies
traindata['FinelineNumber'] = traindata['FinelineNumber'].astype('object')
finelineDummies = pd.get_dummies(traindata[['FinelineNumber']],prefix='FLN')
#Get Weekday Dummies
tmp_WD = pd.get_dummies(traindata[['Weekday']],prefix='WD')
#Get dummies for upcfirst
UF = pd.get_dummies(traindata[['upcfirst']],prefix='uf')

#Top N Fine line numbers
sumFLN = np.sum(finelineDummies,axis=0)
topN_FLN = sumFLN.sort_values(axis=0,ascending=False)[0:1500]
print("FLN:" , len(sumFLN))



FLN: 486


In [87]:
#Add UPC Dummies
#3, 4, 5, 6, 7
list1=['1200000131', '1200000159', '1450001098', '1590013401', '1820000243', '1820053030', '20104500000', '20108700000', '20108800000', '20112300000', '20154200000', '20154500000', '20184400000', '20382100000', '20395900000', '20501200000', '20553800000', '20554600000', '20554800000', '20577400000', '20580000000', '20589000000', '20608200000', '20609300000', '20609900000', '20692800000', '20867000000', '20966500000', '22003200000', '22455600000', '22456500000', '22500200000', '22500500000', '22500800000', '22500900000', '22501100000', '22555500000', '22592200000', '22592300000', '22607600000', '22616700000', '22628200000', '2265571555', '22763000000', '25088200000', '2610000573', '2610000575', '2610000578', '2820000384', '2820031042', '30169183302', '30169183402', '30169183702', '3082', '30869110610', '3107', '3151', '3338320027', '3338360002', '3338365020', '3338365101', '3338366602', '3400000241', '3400040551', '35310020010', '36461304562', '4000042431', '4011', '4016', '4046', '4048', '4062', '4065', '4078', '4082', '4087', '4093', '4159', '4182080791', '4182080793', '4182080795', '4664', '4693', '4827', '4889', '4900000044', '4900000045', '4900000764', '4900000977', '4900001916', '4900002620', '4900004086', '4900005010', '4900005015', '4900005115', '4958', '60538802294', '60538802945', '60538812236', '60538812237', '60538819035', '60538862097', '60538862128', '60538862129', '60538871457', '60538871461', '60538887953', '60538896309', '61126999100', '65876136801', '65876136931', '68113106589', '68113106754', '68113106755', '68113106758', '68113106761', '68113106762', '68113107095', '68113107862', '68113107939', '68113109896', '68113132352', '68113132353', '68113132893', '68113132894', '68113132896', '68113132897', '68113132946', '68113138804', '68113152930', '68113163351', '68113163352', '68113163353', '68113168757', '68113169367', '68113176760', '68113176761', '68113189617', '68113191697', '68113191698', '68113191699', '68113191702', '69922162117', '7084781116', '7100703113', '7100754116', '71458100088', '7225003706', '7225003712', '7225004319', '7294560136', '73114965337', '73114973851', '7343500006', '7373107107', '7432309090', '7442602000', '7494109822', '7539100470', '7539100878', '7618316363', '7763304547', '7763304737', '78035378403', '7874201228', '7874203952', '7874209728', '7874211433', '7874212310', '7874214813', '7874222375', '7874235186', '7874235187', '7874235188', '7874235191', '7874235200', '7874235201', '7874243458', '7874263939', '7874298393', '7874298678', '7874298690', '7874298761', '7932265549', '7976503128', '79936663191', '79936663195', '79936675301', '8066095615', '83032400508', '83032400526', '83032400641', '83032400696', '83032400783', '8520000114', '8520000125', '8520000126', '8520000127', '85303000210', '9147597000']

#8, 9, 12, 14, 15 
list2= ['1070080727', '1200000129', '1200000131', '1200000159', '1200001311', '1200013027', '1200014143', '1200080994', '1200081331', '1254661229', '1254661531', '1254661959', '20108800000', '20966500000', '2200000484', '2200000488', '2200000512', '2200000666', '2200000838', '2200000891', '2200000898', '2200000899', '2410057467', '2430078106', '2430083561', '2430083566', '2430083938', '2500000035', '2800000820', '2800000824', '2800011888', '2800027249', '2800027525', '2800029505', '2800032710', '2800050381', '2800056714', '2840000152', '2840004176', '2840016190', '2840016199', '2840016201', '2840016207', '2840028204', '2840042052', '2840042054', '2840042061', '2840042071', '2840042073', '2840043143', '2840043318', '2840043393', '2840043503', '2840043774', '3040077852', '3040079000', '30521500700', '3107', '3151', '3207', '3338311000', '3338320027', '3338365020', '3400000007', '3400000229', '3400000239', '3400000240', '3400000241', '3400000246', '3400000440', '3400000480', '3400040551', '3485600408', '3800059099', '4000000032', '4000000263', '4000000432', '4000000460', '4000029480', '4000042431', '4000047652', '4011', '4016', '4022', '4045', '4046', '4050', '4051', '4056', '4065', '4087', '4127197110', '4135', '4223830241', '4380700032', '4400003211', '4400003214', '4460031208', '4460031259', '4460031301', '4900000044', '4900000045', '4900000371', '4900000764', '4900000790', '4900000977', '4900001278', '4900001463', '4900001679', '4900001801', '4900001916', '4900002620', '4900002890', '4900003165', '4900004086', '4900005010', '4900005015', '4900005025', '4900005115', '4900005537', '5000021667', '5200032866', '5200032868', '5400010183', '60538819035', '60538862097', '60538871457', '60538871461', '61126910171', '61126911357', '61126954601', '61126981899', '61126999100', '61696007823', '64312604011', '64541671806', '68113102889', '68113104849', '68113107095', '68113107862', '68113109896', '68113157393', '68113170247', '6827473529', '7084781116', '71533943000', '72201100004', '7225004319', '72957184448', '73651154066', '7432309750', '7464100287', '7464100427', '75322200110', '75752800879', '7618316363', '7618316364', '7618316850', '7667710033', '7800008240', '7800008340', '7800009240', '78616200151', '7874201456', '7874201839', '7874203524', '7874203952', '7874204880', '7874205590', '7874205776', '7874208830', '7874208940', '7874209728', '7874211433', '7874235186', '7874235187', '7874235188', '7874235191', '7874235200', '7874235201', '7874235202', '7874243019', '79487801200', '79487801870', '7976503128', '8265750406', '83032400419', '83032400439', '83874102642', '8390000575', '84008511234', '87126040032', '9147586000', '9147593097', '9506', '980000760', '980000761', '980012301', '980080005', '9870']

#18, 19, 20, 21
list3 = ['1034390116', '1200000131', '1200000452', '1380321600', '1380323338', '1380324731', '1380326121', '20966500000', '2140056026', '2200000484', '2200000899', '2600000304', '2600000562', '2600001322', '2622946016', '2622977070', '2622978156', '2643854450', '2775501079', '2775501313', '2840028204', '2840043143', '2840043393', '2899521479', '2899521884', '2899521885', '2903337718', '3150825967', '3218753206', '3338320027', '3399102098', '3400000221', '3400000239', '3400000246', '3600032552', '3650009396', '3650009980', '3742331041', '3837201029', '3980010797', '3989714671', '4000000432', '4000000460', '4000042431', '4011', '40550249187', '4078', '4133382501', '4238598011', '4400003214', '4816801621', '489701408059', '4900000044', '4900000045', '4900000764', '4900000790', '4900000977', '4900001801', '4900001916', '4900002620', '4900003165', '4900004086', '4900005010', '5113164596', '5159632130', '5400075130', '60538804738', '60538804740', '60538862097', '60538862128', '60538862129', '61126981899', '61126999100', '61257218040', '61257218623', '61257219619', '61965909416', '6211886773', '63050925387', '63050925817', '65269548162', '65356928978', '65356976931', '66068504096', '68113105630', '68113107862', '68113107867', '68113179201', '68113179202', '68113189617', '695973620016', '7038215138', '7038264104', '7061210160', '7061240320', '7084781116', '7088111802', '7109160010', '7161100772', '7161193653', '7166200024', '7181506328', '7181506548', '7192414521', '7192414977', '7192414991', '7192414995', '7192415005', '7192454526', '7198074003', '7198090256', '72201100004', '7339001079', '7339001404', '7365084641', '73651154066', '73785779004', '7413000150', '7413000181', '7413000182', '7413056154', '7431974011', '7431974072', '7432309750', '7450990841', '7450990947', '7453490164', '7466000930', '7466010049', '75116662710', '7535307841', '7535310216', '7535310230', '7535310386', '7535310467', '7535310949', '7535311723', '7535311728', '7535311765', '7535311832', '7535311876', '7535312268', '75752800879', '7617180393', '76594016785', '7666616523', '7666616526', '7771175530', '7825730730', '7874201578', '7874210460', '7874235186', '7874235187', '7874235200', '7874235201', '7911868806', '7919100094', '7919100095', '7919100096', '7919123084', '7919123110', '7919123111', '79316591164', '79749686563', '79749687157', '79749687167', '7994600660', '7994605367', '80521910470', '80521991315', '80521991320', '82065010962', '82513102368', '82720410582', '8356805000', '83848745002', '84023710012', '84338801817', '8449511772', '88492010002', '88492010201', '88492010404', '88496298362', '88563180596', '88590957535', '88611160138', '88611178780', '88622791367', '88634500050', '88711132643', '88711196804', '88727600714', '88796105703', '88846209853', '88846209883', '890800513400', '910002645', '910038261', '980000761', '980080005']

#22, 23, 24, 25 
list4 = ['1112011203', '1112019462', '1112019909', '1112022111', '1200000129', '1200000131', '1200000159', '1200001311', '1254600638', '1254661229', '1254661267', '1254661531', '1254661959', '1750020941', '1864346102', '1980003674', '20108700000', '20108800000', '20154200000', '20154500000', '20577400000', '2075306209', '20966500000', '2120057235', '2200000484', '2200000666', '2200000891', '2200000899', '2214345488', '22501100000', '22862200000', '2340000063', '2340000655', '2519222231', '2569500819', '2569504306', '2569504309', '2724284106', '2840028204', '2840042054', '2840042073', '2840043143', '2840043318', '2840043393', '2840043789', '3040077852', '30521500700', '3087812940', '3338311000', '3338320027', '3338365020', '3400000221', '3400000229', '3400000239', '3400000240', '3400000241', '3400000246', '3400000440', '3400000480', '3400008803', '3400013851', '3400040551', '3408663825', '3408667040', '3426441601', '3426443770', '3429904588', '3429904589', '3500055100', '3596850083', '3800057608', '4000000032', '4000000263', '4000000405', '4000000432', '4000042431', '4000047652', '4011', '4016', '4022', '4046', '4048', '4051', '40551150340', '4062', '4065', '4078', '4087', '4093', '4111600592', '4133382501', '4178900121', '4223830241', '4266610708', '4400003210', '4400003211', '4400003213', '4400003214', '4400003412', '4468134589', '4468134627', '4889', '4900000044', '4900000045', '4900000764', '4900000790', '4900000977', '4900001463', '4900001679', '4900001801', '4900001916', '4900002620', '4900002890', '4900003165', '4900004086', '4900005010', '4958', '5200032555', '60538804753', '60538816336', '60538862097', '60538862128', '60538862129', '60538871457', '60538871461', '61126981899', '61126999100', '6164890621', '66553830610', '66572105686', '66572105763', '66572107784', '68113104849', '68113104850', '68113107862', '68113118442', '68113146701', '68113169401', '68113189617', '6827473529', '692249579995', '69922162117', '7084781116', '7279903542', '7314906279', '7314914769', '7314918478', '7314918518', '7314924708', '7314929309', '7314929329', '7314945922', '7314949783', '7314994004', '7339001079', '7339001393', '7432309750', '7535311723', '75680910494', '75680910528', '75680910530', '75680930996', '75680930997', '75680930999', '75752800879', '78035378403', '7874203952', '7874211433', '7874222803', '7874235186', '7874235187', '7874235193', '7874235200', '7874235201', '7874235296', '7874298393', '79357380941', '7940056430', '7962507189', '80240400445', '80240400446', '80240400518', '80240400562', '81157101357', '8265750406', '8390000575', '83972400777', '83972400839', '83972400889', '84110100530', '8520000125', '87897100000', '88309624204', '88359940299', '88590965013', '88859694414', '896400099840', '980000760', '980012301', '980080005']

#26, 27, 28, 29, 
list5 = ['1070002440', '1142383801', '1142394116', '1200000131', '1254661267', '1404532418', '1404532423', '1675121895', '1675152090', '1680043700', '1822200507', '2149601555', '2149601610', '2200000484', '2200000898', '2200000899', '2238496039', '2253204973', '2253205073', '2253206661', '2253209752', '2253212284', '2253213629', '2253213637', '2471911200', '2471911531', '2471920051', '2471920544', '2471921445', '2471921636', '2471922120', '2471930122', '2471930196', '2471930201', '2471930324', '2471930366', '2471930416', '2471930428', '2471930480', '2471930533', '2471930587', '2471930686', '2471930701', '2471930733', '2471930743', '2471930765', '2471930803', '2471930816', '2471930922', '2471930943', '2471930946', '2471930950', '2471930969', '2471930979', '2471930995', '2471931068', '2471931184', '2471931236', '2471931237', '2471931241', '2471931803', '2471931814', '2471932022', '2471932112', '2615691140', '2704572927', '2840042073', '29090500000', '29095500000', '29096900000', '30521500700', '3207', '3224711307', '3224734514', '3224734524', '3224755783', '3224756523', '3224756783', '3224785525', '3224786525', '3278458670', '3278458674', '3338320027', '3338365322', '3400000229', '3400000241', '3408663825', '3686512231', '3686512250', '3686524511', '3867505389', '3867505391', '3877504206', '4000000432', '4000042431', '4011', '4046', '4048', '4050', '4062', '4065', '4078', '4087', '4093', '4111601118', '4127197110', '4133343001', '4223830241', '4317175156', '4338829647', '4380700032', '4460031208', '4460031259', '4460031301', '4488231028', '4488261010', '4601345495', '4601345496', '4601346068', '4601346069', '4656117978', '4687858684', '4889403538', '4889403947', '4900000044', '4900000045', '4900000764', '4900000790', '4900000977', '4900001463', '4900001679', '4900001916', '4900002620', '4900003165', '4900004086', '4900005010', '4958', '5638908313', '60538807104', '60538862097', '60538862128', '60538862129', '60538871457', '64312604011', '68113102659', '68113102667', '68113102672', '68113104849', '68113107060', '68113107061', '68113107862', '68113189617', '71533909020', '71533909067', '71533916001', '71533921001', '71533943000', '71533980040', '7164524514', '7164585025', '72398700524', '72450425000', '7356100123', '74107404238', '7432309750', '75322200110', '7535303660', '75623374123', '75752800879', '7617411921', '7666623141', '7792407393', '7825730140', '7825730142', '7825730404', '7825730838', '7825731868', '7825731869', '7825731904', '7874205776', '7874211433', '7874235186', '79450475963', '79450475973', '7959416257', '81271301903', '83646700061', '83646700075', '83646700114', '84008511234', '84008511238', '84101516021', '84101516022', '85735000346', '88546810016', '88678300140', '88678300142', '88678300195', '9292670900', '9826201465']

#30, 31, 32, 33, 
list6 = ['1090000015', '1200000159', '1200001311', '1200540463', '1254661959', '1500000411', '1500000417', '1500000418', '1500000427', '1500000431', '1500000434', '1500000435', '1500000447', '1500007000', '1500007001', '1500007003', '1500007004', '1500007600', '1500007601', '1500007602', '1500007603', '1500007604', '1500007605', '1500007606', '1500007607', '1500007608', '1500007609', '1500007610', '1500007640', '1500007642', '1500007643', '1500007665', '1500007669', '1500007671', '1500007673', '1500007675', '1500007679', '1500007681', '1500007683', '1500007685', '1500014000', '1920075352', '1980003663', '1980003667', '1980031788', '20108800000', '20154500000', '20966500000', '2120057235', '2340000063', '2840042054', '2840042073', '2840043318', '2840043393', '3040021640', '3040077387', '3040077852', '3040079000', '30521500700', '3338320027', '3338365020', '3400000239', '3500005364', '3500039013', '3500039131', '3500039371', '3500039372', '3500049110', '3500053031', '3500053032', '3500053058', '3500053096', '3600043247', '3600043501', '3600043504', '3663202760', '3700010776', '3700012769', '3700012786', '3700022205', '3700050579', '3700051051', '3700086078', '3700086128', '3700086129', '3700086514', '3700086855', '3700086881', '3700086882', '3700087087', '3700087095', '3700087097', '3700088190', '3700088234', '3700089074', '3700089131', '3700089399', '3700089713', '3700089714', '3700089716', '3700095145', '4000042431', '4000047652', '4011', '4046', '4062', '4078', '4087', '4129440112', '4178900125', '4400003211', '4400003214', '4460030769', '4460030770', '4460030776', '4460030783', '4460030784', '4460030786', '4650015474', '4650073333', '4650073339', '4650073341', '4650073343', '4900000044', '4900000045', '4900000764', '4900000977', '4900001278', '4900001916', '4900002620', '4900002890', '4900003165', '4900005010', '5000021667', '5000037765', '5000062400', '5400010060', '5400013342', '5400015029', '60538862097', '60538862128', '60538871457', '60538871461', '61021464009', '61126981899', '61126999100', '61696001310', '61696007823', '61696010355', '6233878048', '6233887979', '67811210179', '67811244415', '67811256544', '68113102888', '68113102889', '68113107862', '68113109035', '6827473529', '68333351133', '69922162117', '7007453360', '7007457431', '7007458056', '7007458059', '7084781116', '7092210371', '7109160010', '73528240029', '73528240084', '7432309090', '7432309750', '75752800879', '7874201184', '7874201185', '7874203951', '7874203952', '7874204017', '7874204018', '7874204020', '7874204022', '7874204025', '7874204026', '7874204028', '7874208242', '7874208826', '7874209348', '7874211433', '7874235186', '7874235187', '7874235200', '7874235201', '7976503128', '79936606796', '79936617481', '79936629905', '8265750406', '88727668978', '9518801128', '980012301']

#34, 35, 36, 37
list7 = ['1200001311', '1200080994', '1200080996', '1284241001', '1500014000', '1590013401', '20154200000', '20154500000', '20966500000', '2200000488', '2200000899', '22003200000', '22093300000', '22591100000', '22592200000', '22616700000', '22763000000', '2575100988', '2840000289', '2840028204', '2840033579', '2840043143', '2840043774', '30521500700', '3082', '3107', '3112', '3151', '3338311000', '3338311010', '3338311502', '3338320027', '3338320030', '3338325059', '3338340010', '3338353010', '3338353050', '3338360002', '3338360051', '3338360152', '3338365020', '3338365101', '3338365162', '3338365322', '3338366000', '3338366602', '3338370001', '3400000239', '3400000241', '3400000246', '3421', '3710267500', '3710268600', '3800057608', '4000000432', '4000042431', '4011', '4016', '4017', '4020', '4022', '4023', '4029', '4030', '4032', '4036', '4040', '4044', '4045', '4046', '4048', '4050', '4051', '4056', '4062', '4065', '4066', '4067', '4069', '4072', '4073', '4078', '4080', '4082', '4087', '4089', '4093', '4131', '4135', '4159', '4178900105', '4178900121', '4178900125', '4235', '4312', '4378', '4400003202', '4400003214', '4409', '4416', '4499', '4611', '4612', '4663', '4664', '4688', '4693', '4827', '4889', '4900000044', '4900000764', '4900000977', '4900001278', '4900001463', '4900001916', '4900002468', '4900002620', '4900002890', '4900002892', '4900003165', '4900005010', '4900005015', '4900005025', '4958', '4959', '5200012196', '5200012937', '5200020806', '5200032213', '5200032555', '5200032673', '5200033875', '5200033876', '5200033877', '60538800144', '60538862097', '60538862128', '60538871457', '60538871459', '60538871461', '61126999100', '65876136801', '68113104849', '68113107862', '68113109133', '68113132886', '68113132894', '68113132946', '68113178113', '6827473441', '6827473529', '68896200806', '69922162117', '7084781116', '7104113636', '7143000752', '7146410017', '7146418095', '71575610003', '71752413100', '7224013381', '7225003706', '7225003712', '7294560136', '73447533423', '7432309090', '75166677005', '7539100470', '75752800879', '76101040154', '76163520200', '76163520360', '7800001305', '7800008216', '78035378403', '7874201228', '7874203524', '7874203952', '7874209728', '7874211433', '7874212310', '7874222803', '7874235186', '7874235187', '7874235191', '7874235192', '7874235193', '7874235200', '7874235201', '7874298393', '7976503128', '8265750406', '8265755561', '87489600557', '8768400107', '88828940068', '88828940186', '88828940302', '9518801128', '980012301']

#38,39,40,41
list8 = ['1200001311', '1200080994', '1450001098', '1450001100', '1500014000', '1590013401', '1600048366', '18778700092', '20154200000', '20154500000', '20577400000', '20966500000', '2100061223', '2100061526', '2120057235', '22616700000', '22763000000', '2430004102', '2460001001', '2500001970', '2500004772', '2733100616', '2840000210', '2840000288', '2840000289', '2840023987', '2840033579', '2900000822', '3040077852', '3040079000', '30521500700', '3082', '3107', '3151', '3338311000', '3338320027', '3338340010', '3338353010', '3338360002', '3338365020', '3338365322', '3338366602', '3400008803', '3400040551', '3421', '3663200991', '3663202725', '3663202760', '3663203646', '3663205590', '3700092976', '3800039122', '3800057608', '4000042431', '4011', '4016', '4022', '4029', '4032', '4044', '4045', '4046', '4048', '4050', '4051', '4062', '4065', '4069', '4072', '4078', '4082', '4087', '4093', '4135', '4178900105', '4178900115', '4178900121', '4178900125', '4178900211', '4235', '4312', '4400000057', '4400003113', '4400003202', '4400003211', '4400003214', '4400003219', '4400003442', '4460030770', '4460030776', '4664', '4693', '4827', '4889', '4900000044', '4900000764', '4900000977', '4900001278', '4900001463', '4900001916', '4900002890', '4900003165', '4900004575', '4900005010', '4900005015', '4900005025', '4958', '5200032213', '5200032555', '5200032673', '5200033875', '5200033876', '5200033877', '5210037160', '5400010183', '5410722101', '60538800120', '60538800144', '60538862097', '60538862128', '60538871457', '60538871459', '60538871461', '60538871462', '65876136801', '68113107862', '68113132894', '68113132896', '68113132946', '68113174355', '68113178113', '6827473441', '6827473529', '69922162117', '7047029423', '7084781116', '7104113636', '7109160010', '71514111356', '71514172928', '7225003706', '7225003712', '7282017092', '7294560136', '7342000011', '7432308224', '7432309090', '7432309119', '7432309750', '7442602000', '7453490164', '7539100470', '7539100878', '75752800879', '76163520360', '7639703200', '7680000691', '7763304338', '7763304547', '7763304737', '78035378403', '7874201228', '7874201456', '7874203524', '7874203951', '7874203952', '7874204017', '7874204025', '7874204026', '7874209728', '7874211433', '7874214813', '7874222375', '7874222803', '7874222953', '7874235186', '7874235187', '7874235188', '7874235189', '7874235190', '7874235191', '7874235192', '7874235193', '7874235200', '7874235201', '7874235202', '7874235296', '7874243167', '7874298393', '7976503128', '8000000674', '81547301182', '81547301188', '8265750406', '8768400107', '88491200391', '88828940068', '9506', '9518801128', '9518802128']

#42, 43, 44, 999
list9 = ['1112019909', '1200000131', '1200000159', '1200001311', '1200080994', '1300000466', '1564568840', '1680043700', '1864346102', '20108700000', '20108800000', '20154200000', '20154500000', '20382100000', '20577400000', '20966500000', '2120057235', '2127633306', '22501100000', '2264380395', '2471930946', '2471930952', '2529360039', '2569500819', '2643854450', '2773602296', '2800000824', '2840000289', '2840042054', '2840042073', '2840043143', '2840043318', '2840043393', '3018291829', '3040077387', '3040077852', '3040079000', '30521500700', '3218753206', '3224734514', '3338320027', '3338365020', '3400000239', '3400000240', '3400000647', '3400008803', '3408663825', '3429904589', '3650009396', '3700050579', '4000000431', '4000042431', '4000047652', '4011', '4022', '4029', '4045', '4046', '4048', '4050', '4051', '4062', '4065', '4078', '4087', '4133382501', '4133384401', '4138309029', '4138309036', '4138309073', '4138309074', '4140503670', '4400003211', '4400003214', '4460030770', '4460031259', '4460031301', '4650073175', '4889', '4900000044', '4900000045', '4900000764', '4900000790', '4900000977', '4900001278', '4900001463', '4900001679', '4900001916', '4900002890', '4900003165', '4900004086', '4900005010', '4900005015', '5200032866', '60538807733', '60538807734', '60538809877', '60538812235', '60538812238', '60538816335', '60538816336', '60538819035', '60538862097', '60538862128', '60538862129', '60538871457', '60538871461', '60538896309', '61126981899', '61126999100', '64541671806', '66572105686', '6700000850', '68113102615', '68113102889', '68113104849', '68113107862', '68113107867', '68113107941', '68113111868', '68113132353', '68113133018', '68113147314', '68113152929', '68113163351', '68113174202', '68113178113', '68113178251', '68113178252', '68113178253', '68113180649', '68113189617', '68113190428', '6827473529', '69922162117', '7084781116', '7104113636', '7109160010', '71533916001', '71533943000', '7166261224', '72201100004', '7225003706', '7225004319', '7294560136', '72957184448', '72974780278', '7314918518', '73528240084', '7432309090', '7432309750', '7535311723', '75680910528', '75752800879', '76163520360', '7666616523', '7666621383', '7682804702', '7763304737', '78035378403', '7825731254', '7825731574', '7874203524', '7874203641', '7874203952', '7874205776', '7874208674', '7874208678', '7874211433', '7874222803', '7874235186', '7874235187', '7874235188', '7874235191', '7874235200', '7874235201', '7874235202', '7874298393', '7874298620', '7874298761', '7962507258', '7976503128', '80240400446', '80240400518', '80521910470', '80521991331', '87458603436', '87458603481', '87458603500', '87458603510', '87458603514', '87458603516', '87458603520', '87458604270', '88796124402', '9518801128', '9729801382', '980012301', '980080005', '9853']

upcsformodel = (set(list1+list2+list3+list4+list5+list6+list7+list8+list9))
print(len(upcsformodel))

mylist = list(traindata['Upc'])
mylist = [float(i) for i in mylist]
upcsformodel = [float(i) for i in upcsformodel]

A = np.array(mylist)
B = np.array(upcsformodel)
mask = np.invert(np.in1d(A,B))
np.place(A, mask, [None])
K = pd.DataFrame(A)
K.columns=["upcs"]
K['upcs'] = K['upcs'].astype(np.str_)

G1 = pd.get_dummies(K)
G1.drop('upcs_nan', axis=1, inplace=True)
print(G1.shape)



1083
(647054, 1083)


In [88]:
#Create dataframe for model training
X = pd.DataFrame()
X[['VisitNumber','TripType','ScanCount']]=traindata[['VisitNumber','TripType','ScanCount']]

deptColNames = deptDummies.columns
X[deptColNames] = deptDummies
X[UF.columns] = UF
X[topN_FLN.index.values] = finelineDummies[topN_FLN.index.values]
X[G1.columns] = G1 #UPC dummies
X[tmp_WD.columns] = tmp_WD #Weekday dummies

X = X.groupby(['VisitNumber','TripType'],axis=0,as_index=False).sum()
#Add 2-Ngrams
X[df2gram.columns] = df2gram

y = np.ravel(X[['TripType']])  #To avoid the warning thrown by rf.fit
X.drop('TripType', axis=1, inplace=True) #drop the column 

X.drop('Dept_HEALTH AND BEAUTY AIDS',axis=1,inplace=True) #workaround to avoid error for final test file

In [69]:
testdata = pd.read_csv("test.csv")
print(testdata.shape)
testdata = testdata.fillna(0) ## Impute 0
testdata['DepartmentDescription'] = testdata['DepartmentDescription'].replace(to_replace=["MENS WEAR"],value="MENSWEAR")
testdata['DepartmentDescription'] = testdata['DepartmentDescription'].replace(to_replace=["OPTICAL - FRAMES"],value="OPTICAL - LENSES")


(653646, 6)


In [70]:
#UPC First digit
testdata['Upc'] = testdata['Upc'].astype(np.str_)
listupc = list(testdata['Upc'])
testdata['upcfirst'] = [x[0] for x in listupc]
print(len(pd.unique(testdata.upcfirst.ravel())))

#N2Grams for test UPC
N2grams = testdata[['VisitNumber','Upc']]
N2gramsgrp = N2grams.groupby(['VisitNumber'])['Upc'].apply(lambda x: "{%s}" % ', '.join(x)).to_frame()

#print(ng2grp['Upc'])
corpusarray = list(N2gramsgrp['Upc'])
vectorizer = TfidfVectorizer(tokenizer=None, 
                             use_idf=False,norm=None,
                             max_features = 300,ngram_range=(2,2)) 
test_data_features = vectorizer.fit_transform(corpusarray)   
test_data_features = test_data_features.A #toarray()  
vocab = vectorizer.get_feature_names()
df2gram = pd.DataFrame()
df2gram = pd.DataFrame(data=test_data_features,columns=vocab)

#Bin fineline numbers
df = testdata['FinelineNumber'].to_frame()
binsize = 20
labels = [ "{0}".format(i+10) for i in range(0, 10000, binsize) ]
testdata['FinelineNumber'] = pd.cut(df.FinelineNumber, range(0, 10000+binsize, binsize), right=False, labels=labels)


10


In [71]:
#Department dummies
testDeptDummies = pd.get_dummies(testdata[['DepartmentDescription']],prefix='Dept')
testDeptColNames = testDeptDummies.columns


#Get Fineline Dummies
testdata['FinelineNumber'] = testdata['FinelineNumber'].astype('object')
testFLNDummies = pd.get_dummies(testdata[['FinelineNumber']],prefix='FLN')

#Get Weekday Dummies
testWD = pd.get_dummies(testdata[['Weekday']],prefix='WD')

#Get dummies for upcfirst
UF = pd.get_dummies(testdata[['upcfirst']],prefix='uf')

sumFLN = np.sum(testFLNDummies,axis=0)
testTopN_FLN = sumFLN.sort_values(axis=0,ascending=False)[0:1500]
#print(topN_FLN)
print(len(sumFLN))


mylist = list(testdata['Upc'])
mylist = [float(i) for i in mylist]
upcsformodel = [float(i) for i in upcsformodel]

A = np.array(mylist)
B = np.array(upcsformodel)
mask = np.invert(np.in1d(A,B))
np.place(A, mask, [None])
K = pd.DataFrame(A)
K.columns=["upcs"]
K['upcs'] = K['upcs'].astype(np.str_)

G1 = pd.get_dummies(K)
G1.drop('upcs_nan', axis=1, inplace=True)
print(G1.shape)


485
(653646, 1079)


In [72]:
#Create dataframe for model testing (Final test set)
XT = pd.DataFrame()
XT[['VisitNumber','ScanCount']]=testdata[['VisitNumber','ScanCount']]
XT[testDeptColNames] = testDeptDummies

XT[testTopN_FLN.index.values] = testFLNDummies[testTopN_FLN.index.values]

#XT[dftest.columns] = dftest[dftest.columns]

XT[testWD.columns] = testWD
XT[UF.columns] = UF
XT[G1.columns] = G1

XT = XT.groupby(['VisitNumber'],axis=0,as_index=False).sum()

XT[df2gram.columns] = df2gram


In [73]:
XT.shape

(95674, 1949)

In [74]:
dropFromTrain = [x for x in X.columns if (x not in XT.columns)]
print("columns that are in train but not in test") 
print(len(dropFromTrain))

print("columns that are in test but not in train")
#dropFromTest = [x for x in X.columns if (x not in X_train.columns)]
dropFromTest = [x for x in XT.columns if (x not in X.columns)]
print(len(dropFromTest))

columns that are in train but not in test
95
columns that are in test but not in train
90


In [75]:
print(list(dropFromTest))
print(list(dropFromTrain))

['FLN_8690', 'FLN_6290', 'FLN_5690', '1200001311 4011', '1200080994 1200080994', '18778700092 18778700092', '20154200000 68113107862', '20154500000 20154500000', '2471930943 2471930943', '2800011888 7874235186', '2800027249 7874235188', '2800050381 7874235186', '3107 3107', '3338311000 3338311000', '3338320027 7874235186', '3338365020 3151', '3800000127 7874235188', '4011 1200001311', '4011 1284241001', '4011 22003200000', '4011 3338311000', '4011 4016', '4011 4067', '4011 4135', '4011 4178900121', '4011 68113132894', '4011 68113132946', '4011 6827473529', '4011 7442602000', '4011 7453490164', '4011 7874298393', '4011 8265750406', '4022 3338320027', '4045 3338320027', '4045 4045', '4067 4011', '4072 4072', '4078 4087', '4087 4889', '4093 60538871457', '4159 4011', '4178900121 4011', '4664 4011', '4693 4693', '4889 60538871461', '4900001463 4900003165', '4900002468 4900002468', '4900003165 7874235187', '4900004574 4900004579', '4959 4011', '5200033875 5200033876', '5200033876 5200033877

In [76]:
X.drop(dropFromTrain,axis=1,inplace=True) #To avoid error for final test file


In [89]:
X_s = X
print(X_s.shape)
X_s.drop('VisitNumber', axis=1, inplace=True) #drop the column 

(95674, 1954)


In [94]:
#Split into training and validation set
#X_train, X_test, y_train, y_test = train_test_split(X_s, y, test_size=.65, random_state=10) 
sss = StratifiedShuffleSplit(y,test_size=.65,random_state=10)
for train_index, test_index in sss:
    X_train, X_test = X_s.iloc[train_index], X_s.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
print(X_train.shape)

(33482, 1953)


In [96]:
#MODEL 1
y_test = preprocessing.LabelEncoder().fit_transform(y_test)
y_train = preprocessing.LabelEncoder().fit_transform(y_train)

dtrain = xgb.DMatrix( X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

param = { 'bst:eta':0.05, 'silent':1, 'objective':'multi:softprob','num_class':38 }
param["min_child_weight"] = 5
param["subsample"] = 0.8
param["scale_pos_weight"] = 1.0
param["silent"] = 1
param["max_depth"] = 7
param["colsample_bytree"]=0.9

param['nthread'] = 4
param['eval_metric'] = 'mlogloss'

evallist  = [(dtrain,'train')]
plst = param.items()
num_round = 1000   

bst = xgb.train( plst, dtrain, num_round, evallist,verbose_eval=False)

ypred = bst.predict(dtest)
print(log_loss(y_test, ypred) )    
#0.65 test size , (1000 rounds) = 0.801704702723

0.801704702723


In [81]:
#Check, which trip type has most errors
ypredlist = [np.argmax(ypred[i]) for i in range(0,len(ypred)) ]
#ypredlist = np.argmax(ypred, axis=1)
sumdf = pd.Series(ypredlist).value_counts().to_frame()
sumdf.reset_index(level=0, inplace=True)
sumdf.columns = ["triptype","predicted"]

tmpdf = pd.Series(y_test).value_counts().to_frame()
tmpdf.reset_index(level=0, inplace=True)
tmpdf.columns = ["triptype","Actual"]

sumdf = pd.merge(sumdf, tmpdf, on='triptype')
sumdf['err']= (sumdf['Actual']-sumdf['predicted'])/sumdf['Actual']
print(np.sum(sumdf['err']))
#(Actual-pred)/Actual
sumdf.sort_values('err')

5.25545722763


,triptype,predicted,Actual,err
1,31,8354,6432,-0.298818
3,32,4980,3985,-0.249686
22,23,463,386,-0.199482
7,17,2739,2404,-0.139351
12,24,1445,1290,-0.120155
0,5,8757,7905,-0.107780
6,2,3228,2985,-0.081407
2,6,6506,6152,-0.057542
15,3,872,830,-0.050602
9,28,2048,1953,-0.048643


In [63]:
#Create and fit the model
#rf = RandomForestClassifier( n_estimators=200, random_state=101)
#rf.fit(X_train,y_train)
#y_score = rf.predict_proba(X_test)

#gbm = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1, max_depth=4, max_features=0.2, random_state=11)
#gbm = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1, random_state=11)
#gbm.fit(X_train,y_train)
#y_score = gbm.predict_proba(X_test)


In [82]:
XT.drop(dropFromTest,axis=1,inplace=True) #workaround to avoid error for final test file
#Rearrange the columns
VisitNum = XT[['VisitNumber']]
XT = XT[X_s.columns]


In [83]:
#Create the model on 100% of training set
y = preprocessing.LabelEncoder().fit_transform(y)


dtrain = xgb.DMatrix( X_s, label=y)
dtest = xgb.DMatrix(X_s)

param = { 'bst:eta':0.05, 'silent':1, 'objective':'multi:softprob','num_class':38 }
param["min_child_weight"] = 5
param["subsample"] = 0.8
param["scale_pos_weight"] = 1.0
param["silent"] = 1
param["max_depth"] = 7
param["colsample_bytree"]=0.9


param['nthread'] = 4
param['eval_metric'] = 'mlogloss'

evallist  = [(dtrain,'train')]
plst = param.items()
#print(plst)
num_round = 500   
bst = xgb.train( plst, dtrain, num_round, evallist,verbose_eval=False)

ypred = bst.predict(dtest)
#print(ypred)
print(log_loss(y, ypred) ) 

0.410908234441


In [84]:
dtest1 = xgb.DMatrix(XT)
y_score = bst.predict(dtest1)
np2 = np.array(VisitNum)
np1 = np.concatenate((np2,y_score),axis=1)
df = pd.DataFrame(np1)

submit = pd.read_csv("sample_submission.csv")
df.columns = submit.columns
df[['VisitNumber']]=df[['VisitNumber']].astype(int) #Changing VisitNumber to integer
df.to_csv("lastone.csv",sep=",",header=True,index=False)